In [ ]:
"""
We collected our data using Bloomberg terminal, including the daily price and daily volume of all S&P 500 constitutents from 11/2002-10/2018.
For sector performance, we use 11 sector index funds'price (S5FINL, S5INFT, S5RLST, S5UTIL, S5ENRS, S5MATR, S5HLTH, S5COND, S5CONS, S5INDU, S5TELS)

The following code shows how we merge, organize and clean different Excel files together to create one single CSV file 
which contains cross-sectional time series information of all S&P 500 constituents and sectors.

"""


In [ ]:
import os
import pandas as pd
import datetime
import openpyxl

In [ ]:
def processEachFile(filepath):
    pathDir = os.listdir(filepath)
    output = None
    for aDir in pathDir:
        thisFuckingFile = os.path.join(filepath, aDir)
        df = processFile(thisFuckingFile)
        if output is None:
            output = df
        else:
            output = output.append(df)
        # print(output.head())
    output.to_csv('processData.csv')
    output = processReturn(output)
    output.to_csv('processReturn.csv')
    return output



In [ ]:
def processReturn(data):
    df = None
    processData = data.sort_values(['Date'])
    processData.index = range(len(processData))
    equityNames = data.Ticker.unique()
    print('There are', str(len(equityNames)), 'to process.')
    for name in equityNames:
        print('I am process', name)
        nameData = data[data.Ticker == name]
        processDate = nameData['Date']
        processDate = processDate.diff()
        processInd = list(processDate.index[processDate >= datetime.timedelta(days = 30)])
        if len(processInd) > 0:
            processStart = 0
            processInd.append(len(processInd) - 1)
            for i in range(len(processInd)):
                processSubNameData = nameData[processStart : processInd[i]]
                processPrices = processSubNameData['Price']
                processReturn = processPrices.pct_change()
                processSubNameData['Return'] = processReturn
                if df is None:
                    df = processSubNameData
                else:
                    df = df.append(processSubNameData)
                processStart = processInd[i]
        else:
            processPrices = nameData['Price']
            processReturn = processPrices.pct_change()
            nameData['Return'] = processReturn
            if df is None:
                df = nameData
            else:
                df = df.append(nameData)
    return df


In [ ]:
def processFileName(fileName):
    subName = fileName.split('\\')
    nameList = subName[-1].split(' ')
    temp = nameList[-1].split('.')
    dateString = temp[0][:-1] + nameList[-3] + nameList[-2]
    return datetime.datetime.strptime(dateString, '%Y%b%d')



In [ ]:
def processFile(fileName):
    processFile = openpyxl.load_workbook(fileName)
    processSheet = processFile['Worksheet']
    processTicker = []
    processPrice = []
    processDate = processFileName(fileName)
    processDates = []
    for (ind, processValue) in enumerate(processSheet.values):
        if ind == 0:
            continue
        else:
            try:
                aFuckingTicker = processValue[0]
                aFuckingPrice = float(processValue[-1])
                aFuckingDate = processDate
                processTicker.append(aFuckingTicker)
                processPrice.append(aFuckingPrice)
                processDates.append(aFuckingDate)
            except:
                continue
    df = pd.DataFrame({'Date': processDates, 'Ticker': processTicker, 'Price': processPrice})
    return df




In [ ]:
f = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')
df = pd.read_csv('C:\\Users\\Dian\\OneDrive\\Studying\\Quantitative Finance\\Deep Learning\\Project\\processData.csv', converters={'Date':f})
output = processReturn(df)
output.to_csv('processReturn.csv')

In [ ]:
def match():
    
    def time_convert(time):
        new_time  = datetime.datetime.strptime(time, "%m/%d/%Y").strftime("%Y-%m-%d")
        return new_time
    
    def match_volume(date,ticker):
        vol = volume.loc[date,ticker]
        return vol
    price = pd.read_csv('data.csv',index_col= 0)
    volume = pd.read_csv('volume.csv')
    sector = pd.read_csv('Sector_Info.csv',index_col = 0)
    sector_price = pd.read_csv('Sector Performance.csv',index_col = 0)

    sector_price['Date'] = sector_price['Date'].apply(time_convert)
    volume.iloc[:,0] = volume.iloc[:,0].apply(time_convert)
    volume.index = volume.iloc[:,0]
    volume = volume.iloc[:,1:]

    price['volume'] = price.apply(lambda x: match_volume(x['Date'],x['Ticker']),axis = 1)
    price = price.sort_values(['Ticker','Date'])
    price = price.reset_index(drop = True)
    price_sector = price.merge(sector,how ='left',left_on='Ticker',right_on = 'ticker')

    price_sector = price_sector[['Date','Ticker','Price','volume','GICS']]
    price_sector.columns = ['Date','Ticker','Price','Volume','Sector']
    price_sector.to_csv('price_sector.csv') 
    pass

match()